# 1. Libraries import

In [ ]:
# Import standart libraries
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import mean_absolute_percentage_error
from scipy.optimize import minimize
import sys

sys.path.append('./input/pgs501-lib')
from lib.config import CFG
from lib import holiday, processing

!cp -R ./input/pgs501-lib/data ./
!mkdir ./data/optimal

# 2. Basic settings

In [ ]:
# SOFT prediction assumes the sales are constant after 2018-01-01; HARD prediction assumes the linear trend
SOFT_PREDICTION = True

# Perform CV or not
do_cv = False

# Perform main prediction or not
do_prediction = True

# Prefix to save the data at
prefix = 'soft_prediction' if SOFT_PREDICTION else 'hard_prediction'

# 3. Helper functions for dataframe preparation, prediction and optimization

## 3.1. `prepare_df`
Prepares the dataframe by adding the necessary columns. Returns the modified dataframe, the list of column groups and the reasonable initial guess of the parameters.

- `df` - the input dataframe.
- `countries` - the list of countries to perform the operations on.

In [ ]:
def prepare_df(df, countries):
    # 1. GDP factor
    gdp_factor = processing.get_gdp_factor(CFG.gdp_csv, countries, CFG.years)
    df = df.join(gdp_factor, on=['country', 'year'], how='left')
    
    # 2. Holidays
    hdays_cols = []
    for country in countries:
        daynum, hmap, hds_names = holiday.get_holiday_map(CFG.countries_2l[country], CFG.years)
    
        hnames = [f'hol_{country}_{n}_{hds_names[n - len(hdays_cols)]}' for n in range(len(hdays_cols), len(hdays_cols) + hmap.shape[1])]
        hdays_cols += hnames
    
        hdf = pd.DataFrame(hmap, columns=hnames).reset_index(names='daynum').set_index('daynum')
        df = df.join(hdf, on='daynum', how='left')
        df.loc[df.country != country, hnames] = 0
    
    hdays_cols = [col for col in hdays_cols if 'BEGIN' not in col]
    print(f'Number of holidays = {len(hdays_cols)}')
    
    # 3. Store
    store_cols = []
    for st in CFG.stores:
        df[st] = (df['store'] == st).astype(int)
        store_cols.append(st)
    
    # 4. Products
    product_cols = []
    product_cols_restricted = {
        'Holographic Goose': ['cos t'],
        'Kaggle': ['sin t/2', 'cos t/2'],
        'Kaggle Tiers': ['sin t/2', 'cos t/2'],
        'Kerneler': ['sin t'],
        'Kerneler Dark Mode': ['sin t']
    }
    
    for n, product in enumerate(CFG.products):
        col = f'pr_{n} const'
        df[col] = (df['product'] == product).astype(int)
        product_cols.append(col)
    
        sincos_features = ['sin t', 'cos t', 'sin t/2', 'cos t/2']
        for sc in sincos_features:
            if sc in product_cols_restricted[product]:
                col = f'pr_{n} {sc}'
                df[col] = (df['product'] == product).astype(int) * df[sc]
                product_cols.append(col)
    
    # 5. Weekday
    week_cols = []
    for n in range(4, 7):
        col = f'wd_{n}'
        df[col] = (df['weekday'] == n).astype(int)
        week_cols.append(col)
    
    # 6. Day number
    daynum_cols = ['daynum']
        
    # Column groups
    col_groups = [
        ['gdp_factor'],
        store_cols,
        product_cols,
        week_cols,
        daynum_cols,
        hdays_cols,
    ]

    initial_guess = [
        [-1.9, 0.01], # GDP
        [0.33, 0.33], # Stores
        [0.1 for pc in product_cols[1:]],  # Product cols
        [0 for wd in range(3)], # Weekday
        [3, 1e-2], # ReLU
        [0 for hd in hdays_cols],  # Holidays
    ]

    return df, col_groups, initial_guess

## 3.2. `predict` 
The main function that performs the prediction.

- `coef` - the parameters of the model.
- `x` - input data.
- `round` - optional parameter determines, whether the result is rounded to the nearest int or not.

In [ ]:
def predict(coef, x, round=False):
    cgn, ign = 0, 0
    x_groups = []
    coef_groups = []
    for cg, ig in zip(col_groups, initial_guess):
        x_groups.append(x[:, cgn: cgn + len(cg)])
        coef_groups.append(coef[ign: ign + len(ig)])
        cgn += len(cg)
        ign += len(ig)

    m1 = x_groups[0][:, 0] * coef_groups[0][1] + coef_groups[0][0]
    m2 = x_groups[1] @ np.array((coef_groups[1][0], coef_groups[1][1], 1 - coef_groups[1][0] - coef_groups[1][1]))
    m3 = x_groups[2][:, 0] + x_groups[2][:, 1:] @ coef_groups[2]
    m4 = 1 + x_groups[3] @ coef_groups[3]

    daynum = x_groups[4][:, 0]
    if SOFT_PREDICTION:
        daynum = np.clip(daynum, a_min=0, a_max=2922)
    m5 = 1 + np.clip((daynum / 365 - coef_groups[4][0]) * coef_groups[4][1], a_min=0, a_max=None)
    m6 = 1 + x_groups[5] @ coef_groups[5] # Holidays

    y_pred = m1 * m2 * m3 * m4 * m5 * m6
    return np.round(y_pred).astype(int) if round else y_pred

## 3.3. `get_score`
Returns the MAPE score using current values of parameters

- `coef` - the parameters of the model.
- `x` - input data.
- `y_true` - ground true values of the prediction.
- `round` - optional parameter determines, whether the result is rounded to the nearest int or not.
- `reg` - optional  parameter could be passed that is used for LASSO regularization.

In [ ]:
def get_score(coef, x, y_true, round=False, reg=0):
    y_pred = predict(coef, x, round)
    score = mean_absolute_percentage_error(y_true, y_pred)
    #score += reg * np.sum(np.abs(coef[4: 130]))
    return score

## 3.4. `optimize`
Performs the optimization procedure and returns the object with optimal parameters.
If `opt_groups` is None, optimizes all available parameters. Otherwise only the specified groups of parameters are optimizeed, while others are fixed. This is done in the purpose of performance.

- `coef0` - the initial values of model's parameters.
- `x_train` - train input data.
- `y_train` - ground true values of the train data.
- `reg` - optional  parameter could be passed that is used for LASSO regularization.
- `max_iter` - optional limit of iterations.
- `opt_groups` - the list indicating what groups of parameters to train.

`on_epoch` - helper function that is called after each iteration.

In [ ]:
def on_epoch(intermediate_result):
    #print(f'{intermediate_result.fun}')
    pass


def optimize(coef0, x_train, y_train, reg=0, maxiter=30, opt_groups=None):
    if opt_groups is None:
        # Optimize all
        opt_indices = list(range(len(cols)))
    else:
        # Optimize only the indices from the selected groups
        opt_indices = []
        ign = 0
        for ig_id, ig in enumerate(initial_guess):
            if ig_id in opt_groups:
                opt_indices += list(range(ign, ign + len(ig)))
            ign += len(ig)
    local_coef = coef0.copy()

    def objective1(x):
        return get_score(x, x_train, y_train, reg=reg)

    def objective2(x):
        local_coef[opt_indices] = x
        return get_score(local_coef, x_train, y_train, reg=reg)

    res = minimize(objective1 if opt_groups is None else objective2,
                   x0=local_coef if opt_groups is None else local_coef[opt_indices],
                   callback=on_epoch,
                   options={'maxiter': maxiter, })

    if opt_groups is not None:
        local_coef[opt_indices] = res.x
        res.x = local_coef

    return res

## 3.5. Optimization pipeline (of the 1st level)
Perform the whole optimization cycle. This cycle consists of several optimization steps, at each of which a group of column sets is optimized.

- `df_train` - the train dataframe.
- `df_test` - the test dataframe.
- `cols` - the column groups list.
- `initial_guess` - the initial guess of the parameters.
- `optimization_steps` - the list of optimization steps.
- `lasso_reg` - optional  parameter could be passed that is used for LASSO regularization.
- `print_params` - whether to print params after each step or not.

In [ ]:
def optimization_pipeline(df_train, df_test, cols, initial_guess, optimization_steps, lasso_reg=0, print_params=False):
    x_train = df_train[cols].to_numpy()
    y_train = df_train['num_sold'].to_numpy()
    x_test = df_test[cols].to_numpy()
    y_test = df_test['num_sold'].to_numpy()

    coef0 = np.concatenate(initial_guess)

    for step in optimization_steps:
        res = optimize(coef0, x_train, y_train, reg=lasso_reg, opt_groups=step['groups'], maxiter=step['niter'])
        if print_params:
            print(res.x)
        coef0 = res.x
        score1 = get_score(res.x, x_train, y_train, round=False, reg=0)
        score2 = get_score(res.x, x_train, y_train, round=True, reg=0)
        print(f'\tstep {step["name"]}: {res.fun}\t{score1}\t{score2}')
    train_score = get_score(res.x, x_train, y_train, True)
    test_score = None if np.isnan(y_test).any() else get_score(res.x, x_test, y_test, True)
    return res, (train_score, test_score)

## 3.5. Optimization pipeline (of the 2nd level)
Perform the whole optimization cycle several times, each time with different countries.
This could be useful for training on the subsets of countries.

- `df_train` - the train dataframe.
- `df_test` - the test dataframe.
- `cols` - the column groups list.
- `countries_groups` - the list of countries groups.
- `initial_guess` - the initial guess of the parameters.
- `optimization_steps` - the list of optimization steps.

In [ ]:
def optimization_pipeline2(df_train, df_test, cols, countries_groups, initial_guess, optimization_steps):
    x = []
    df1, df2 = df_train.copy(), df_test.copy()
    df1['prediction'] = 0
    df2['prediction'] = 0
    for countries in countries_groups:
        if len(df2[df2.country.isin(countries)]) == 0:
            continue
        print(f'\tDo optimization on the countries: ' + ', '.join(countries))
        res, (train_score, test_score) = optimization_pipeline(df1[df1.country.isin(countries)], 
                                                               df2[df2.country.isin(countries)], 
                                                               cols, 
                                                               initial_guess, 
                                                               optimization_steps)
        
        df1.loc[df1.country.isin(countries), 'prediction'] = predict(res.x, df1[df1.country.isin(countries)][cols].to_numpy(), True)
        df2.loc[df2.country.isin(countries), 'prediction'] = predict(res.x, df2[df2.country.isin(countries)][cols].to_numpy(), True)
        x.append(res.x)

    res = {'x': x}
    train_score = mean_absolute_percentage_error(df1['num_sold'], df1['prediction'])
    test_score = None if pd.isna(df2['num_sold']).any() else mean_absolute_percentage_error(df2['num_sold'], df2['prediction'])

    return res, (train_score, test_score)

def predict2(coef, df, cols, countries_groups, round=False):
    df1 = df.copy()
    df1['prediction'] = 0.0
    for x, countries in zip(coef, countries_groups):
        df1.loc[df1.country.isin(countries), 'prediction'] = predict(x, df1[df1.country.isin(countries)][cols].to_numpy(), round)
    return df1['prediction']

# 4. Prepare data

In [ ]:
# First, we use external library to add the columns
df = processing.data_read_and_combine(CFG.train_csv, CFG.test_csv)
df = processing.data_add_date_features(df)

# Second, we use local `prepare_df`
df, col_groups, initial_guess = prepare_df(df.copy(), CFG.countries)

# Drop some data for Kenya
#df = df[(df.year > 2012) | (df.country != 'Kenya')]  # Drop Kenya's 2010-2012

# Keep the original DF
df_orig = df.copy()
df_submit = df[(df.test == 1)]
df = df[(df.test == 0) & (~df['num_sold'].isna())]

# THe whole list of the columns
cols = sum(col_groups, [])

# 5. CV

We set the parameters for CV and perform it. The results are saved in `./data/optimal`

In [ ]:
optimization_steps = [
    {'name': 'gspw', 'groups': [0, 1, 2, 3], 'niter': 120},
    {'name': 'daynum', 'groups': [4], 'niter': 100},
    {'name': 'holidays', 'groups': [5], 'niter': 100},
    {'name': 'all', 'groups': None, 'niter': 100},
]

countries_groups = [
    ['Canada', 'Finland', 'Italy', 'Kenya', 'Norway', 'Singapore'],
    ['Kenya'], 
]

if do_cv:
    train_scores, test_scores = [], []
    df['oof'] = None
    for year in CFG.years[:-3]:
        print(f'Year = {year}')

        res, (train_score, test_score) = optimization_pipeline2(df[df.year != year], 
                                                                df[df.year == year], 
                                                                cols, 
                                                                countries_groups, 
                                                                initial_guess, 
                                                                optimization_steps)
        with open(f'./data/optimal/{prefix}_{year}_param.pkl', 'wb') as f:
            pickle.dump(res, f)

        df_orig['pred'] = predict2(res['x'], df_orig, cols, countries_groups, False)
        df_orig.to_csv(f'./data/optimal/{prefix}_{year}_imputation.csv')

        if year > 0:
            df.loc[df.year == year, 'oof'] = predict2(res['x'], df[df.year == year], cols, countries_groups, True)
            train_scores.append(train_score)
            test_scores.append(test_score)
            print(f'{year}: {train_score=} {test_score=}')
        else:
            print(f'Full data: {train_score=}')
            
    print(f'\nAVG: train={np.mean(train_scores)}, test={np.mean(test_scores)}\n\n')
    df.to_csv(f'./data/optimal/{prefix}_result.csv')

# 6. Main prediction
Perform the main prediction. Here, the values for maximum iterations are higher.

At the output, there are two files: 'clear' and 'raw'.

In [ ]:
optimization_steps = [
    {'name': 'gspw', 'groups': [0, 1, 2, 3], 'niter': 120},
    {'name': 'daynum', 'groups': [4], 'niter': 100},
    {'name': 'holidays', 'groups': [5], 'niter': 150},
    {'name': 'all', 'groups': None, 'niter': 150},
]

if do_prediction:
    res, (train_score, test_score) = optimization_pipeline2(df, df_submit, cols, countries_groups, initial_guess, optimization_steps)
    with open(f'submission_param.pkl', 'wb') as f:
        pickle.dump(res, f)
    
    # Prepare submissions
    df_submit['num_sold_raw'] = predict2(res['x'], df_submit, cols, countries_groups, True)
    df_submit.to_csv(f'{prefix}_submission_raw.csv')

    df_submit['num_sold'] = np.round(df_submit['num_sold_raw']).astype(int)
    df_submit[['id', 'num_sold']].to_csv(f'{prefix}_submission_clear.csv', index=False)

## Check, that the beginning and the end of file is okay.

In [ ]:
!head {prefix}_submission_clear.csv

"""id,num_sold
230130,154
230131,1001
230132,759
230133,444
230134,519
230135,313
230136,2029
230137,1540
230138,900""";

In [ ]:
!tail {prefix}_submission_clear.csv

""" For SOFT prediction:
328670,379
328671,2404
328672,2138
328673,1093
328674,1270
328675,448
328676,2840
328677,2527
328678,1291
328679,1501"""

""" For HARD prediction
328670,396
328671,2506
328672,2230
328673,1139
328674,1324
328675,467
328676,2962
328677,2635
328678,1346
328679,1565""";

# Ready to submit!